# Importing Libraries

In [9]:
import pandas as pd
from tensorflow.keras.models import load_model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM,GRU

# Importing DataBase

In [10]:
df_daily = pd.read_csv('/content/drive/My Drive/ML Projects/Pharma Sales/Data/salesdaily.csv')
df_daily = df_daily['N02BE']


# Importing Pre-Treined models

In [12]:
gru_mrf = load_model('/content/drive/My Drive/ML Projects/Pharma Sales/GRU Models/GRU_MostRelavantFeatures.h5') #time_step = 30
gru_mrf_wm = load_model('/content/drive/My Drive/ML Projects/Pharma Sales/GRU Models/GRU_MostRelavantFeatures_weekdayMonth.h5')#time_step = 120
gru = load_model('/content/drive/My Drive/ML Projects/Pharma Sales/GRU Models/GRU_OneFeature.h5')#time_step = 30
gru_wm = load_model('/content/drive/My Drive/ML Projects/Pharma Sales/GRU Models/GRU_weekdayMonth.h5')#time_step = 120
gru_wm_cylic = load_model('/content/drive/My Drive/ML Projects/Pharma Sales/GRU Models/GRU_weekdayMonth_cyclic.h5')#time_step = 120

In [13]:
lstm_mrf_wm = load_model('/content/drive/My Drive/ML Projects/Pharma Sales/LSTM Models/LSTM_multiFeature_mostRelevantFeatures_weekdayMonth.h5')#time_step = 60
lstm = load_model('/content/drive/My Drive/ML Projects/Pharma Sales/LSTM Models/LSTM_oneFeature.h5')#time_step = 30
lstm_wm = load_model('/content/drive/My Drive/ML Projects/Pharma Sales/LSTM Models/LSTM_multiFeature_weekdayMonth.h5')#time_step = 60

#Functions

In [14]:
def dataSplit(df,train_percent,val_percent):
  train_limit = int(round((df.shape[0])*train_percent,0))
  val_limit = int(round((df.shape[0])*(train_percent+val_percent),0))
  
  df_train = df[:train_limit]
  df_val = df[train_limit:val_limit]
  df_test = df[val_limit:]

  print(df_train.shape)
  print(df_val.shape)
  print(df_test.shape)

  return df_train,df_val,df_test

In [15]:
def normalizing_data(df_train,df_val,df_test,frange):
  sc = MinMaxScaler(feature_range = frange)
  try:
    aux = df_train.shape[1]
    df_train_norm = sc.fit_transform(df_train)
    df_val_norm = sc.transform(df_val)
    df_test_norm = sc.transform(df_test)

  except:
    df_train = df_train.values.reshape(-1,1)
    df_val = df_val.values.reshape(-1,1)
    df_test = df_test.values.reshape(-1,1)

    
    df_train_norm = sc.fit_transform(df_train)
    df_val_norm = sc.transform(df_val)
    df_test_norm = sc.transform(df_test)

  print(df_train_norm.max(axis=0))
  print(df_train_norm.min(axis=0))
  print('\n')

  print(df_val_norm.max(axis=0))
  print(df_val_norm.min(axis=0))
  print('\n')

  print(df_test_norm.max(axis=0))
  print(df_test_norm.min(axis=0))

  print('-------------------------')

  print(df_train_norm.shape)
  print(df_val_norm.shape)
  print(df_test_norm.shape)
  return df_train_norm,df_val_norm,df_test_norm

In [16]:
def Regressor(future_steps,time_steps,n_features,df_train,df_val,df_test):
  # future_steps: how many days in the furure to predict 
  # time_steps: number of samples the LSTM model will use for predict 28 days in the future
  # n_features: number of features (LSTM input) 

  #################### TRAIN #################################################
  # Defining the number of samples for the train data
  n_samples = df_train.shape[0]
  x_train = []
  y_train = []

  # shaping data to have the number of time_steps in the x_train ans the 28 future values in the y_train 
  for i in range(time_steps,n_samples - future_steps): 
    x_train.append(df_train[i - time_steps : i, :]) 
    y_train.append(df_train[i : i + future_steps, -1 ]) 

  x_train,y_train = np.array(x_train),np.array(y_train) # transforming in array
  # LSTM need 3D tensor as input 
  x_train = np.reshape(x_train,(x_train.shape[0],x_train.shape[1],n_features))  

  ##################### VALIDATION ###############################################

  df_val = np.concatenate((df_train[-time_steps:],df_val))

  n_samples = df_val.shape[0]
  x_val = []
  y_val = []

  for i in range(time_steps,n_samples - future_steps):
    x_val.append(df_val[i - time_steps : i, :]) 
    y_val.append(df_val[i : i + future_steps,-1]) 

  x_val,y_val = np.array(x_val),np.array(y_val)
  x_val = np.reshape(x_val,(x_val.shape[0],x_val.shape[1],n_features))

  ####################### TEST ################################################

  df_test = np.concatenate((df_val[-time_steps:],df_test))
  n_samples = df_test.shape[0]
  x_test = []
  y_test = []

  for i in range(time_steps,n_samples - future_steps):
    x_test.append(df_test[i - time_steps : i, :]) 
    y_test.append(df_test[i : i + future_steps,-1]) 

  x_test,y_test = np.array(x_test),np.array(y_test)
  x_test = np.reshape(x_test,(x_test.shape[0],x_test.shape[1],n_features))

  x_train = x_train.astype(np.float32)
  y_train = y_train.astype(np.float32)

  x_val = x_val.astype(np.float32)
  y_val = y_val.astype(np.float32)

  x_test = x_test.astype(np.float32)
  y_test = y_test.astype(np.float32)
  
  return x_train,x_test,x_val,y_train,y_val,y_test

# Extracting models weights